In [1]:
from transformers import pipeline
import pandas as pd
import numpy as np

c:\Users\marko.vasilic\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("..\data\smart_store_reviews.csv")
df = df.sample(frac=1).reset_index(drop=True)
new_df = df.iloc[:100, :]
original_text = 'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.'

## BART

- Reasons for chosing bart:
    1. Best summary out of all for same example, he respected grammar, he included the most important points from review.(see the notepad)
    2. He had same result as pegasus
    3. He includes in his training many schemes (more than others) like:
        - Token Masking: Random tokens in a sentence are replaced with [MASK]. The model learns how to predict the single token based on the rest of the sequence.
        - Token Deletion: Random tokens are deleted. The model must learn to predict the token content and find the position where the token was deleted from.
        - Text Infilling: A fixed number of contiguous tokens are deleted and replaced with a single [MASK] token. The model must learn the content of the missing tokens and the number of tokens.
        - Sentence Permutation: Sentences (separated by full stops) are permuted randomly. This helps the model to learn the logical entailment of sentences.
        - Document Rotation: The document is rearranged to start with a random token. The content before the token is appended at the end of the document. This gives insights into how the document is typically arranged and how the beginning or ending of a document looks like.
    4. He uses encoder and decoder
    5. 400 milion parameters

In [3]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
print(summarizer(original_text, max_length=130, min_length=30, do_sample=False))

[{'summary_text': 'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. It is the second tallest free-standing structure in France after the Millau Viaduct.'}]


In [22]:
lengths = df["review_body"].str.len()
argmax = np.where(lengths == lengths.max())[0]
print(argmax)
df.iloc[argmax]
df.iloc[argmax]['review_body']
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
#print(df.iloc[argmax[0]]['review_body'])
print(summarizer(df.iloc[argmax[0]]['review_body'], max_length=100, min_length=30, do_sample=False))

[133018]
[{'summary_text': 'This bag can hold a ton of stuff, and it has so many pockets! Many of the pockets have elastic around the top, to better hold things like bottles and sippy cups. The included travel changing pad is a good size, and larger than it seems in the photos.'}]


In [5]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summarized_reviews = []
for r in new_df['review_body']:
    summarized_reviews.append(summarizer(r, max_length=100, min_length=10, do_sample=False))
new_df['summarized_review'] = [summary[0]["summary_text"] for summary in summarized_reviews] 

Your max_length is set to 100, but you input_length is only 30. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 100, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 100, but you input_length is only 58. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Your max_length is set to 100, but you input_length is only 29. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Your max_length is set to 100, but you input_length is only 32. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 100, but you input_length is only 73. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 100, but you input_length is only 20. You might consider 

In [6]:
print(new_df['summarized_review'][0])
print(new_df['review_body'][0])
new_df

I was really excited about getting this. However, it isn't compatible to my keyboard despite seeming so. Enclosed is pics for comparison.
I was really excited about getting this. However, it isn't compatible to my keyboard despite seeming so. Enclosed is pics for comparison.


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,summarized_review
0,en_0824773,product_en_0722697,reviewer_en_0755840,2,I was really excited about getting this. Howev...,Not applicable,en,pc,I was really excited about getting this. Howev...
1,en_0451671,product_en_0821454,reviewer_en_0028491,4,A little snug but I like them and I will go up...,Four Stars,en,shoes,A little snug but I like them and I will go up...
2,en_0582960,product_en_0067606,reviewer_en_0276540,4,I put both sets in a hanging glass candle hold...,"Pretty and functional, but one stopped working",en,lawn_and_garden,The light is soft and warm. It's not quite bri...
3,en_0015995,product_en_0700961,reviewer_en_0686612,1,I couldn't use this product. The packaging was...,I couldn't use this product. The packaging was...,en,shoes,The packaging was inadequate and the lid arriv...
4,en_0226240,product_en_0932759,reviewer_en_0565732,4,Highly recommend. The story of the youngest br...,Her Monster,en,digital_ebook_purchase,The story of the youngest brother in the crime...
...,...,...,...,...,...,...,...,...,...
95,en_0249743,product_en_0595890,reviewer_en_0413731,3,"The scent is nice, I expected the consistency ...",It's ok,en,beauty,"The scent is nice, I expected the consistency ..."
96,en_0269088,product_en_0587944,reviewer_en_0847314,2,They sent the wrong scent in a beat up package.,wrong thing sent,en,drugstore,CNN.com will feature iReporter photos in a wee...
97,en_0563784,product_en_0778840,reviewer_en_0736076,5,The duffel bag is of good quality but weighs v...,Good bag but heavy,en,luggage,The duffel bag is of good quality but weighs v...
98,en_0771345,product_en_0906567,reviewer_en_0639063,2,Bought this in December and it broke when I op...,Not durable,en,pc,Bought this in December and it broke when I op...


## PEGASUS

In [7]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokens = tokenizer(original_text, truncation=True, padding="longest", return_tensors="pt")
summary = model.generate(**tokens, max_length=100)
tokenizer.decode(summary[0])

'The Eiffel Tower is a landmark in Paris, France.'

In [8]:
lengths = df["review_body"].str.len()
argmax = np.where(lengths == lengths.max())[0]
df.iloc[argmax]
df.iloc[argmax]['review_body']
tokens = tokenizer(original_text, truncation=True, padding="longest", return_tensors="pt")
summary = model.generate(**tokens, max_length=100)
tokenizer.decode(summary[0])
#print(df.iloc[argmax[0]]['review_body'])
print(summarizer(df.iloc[argmax[0]]['review_body'], max_length=100, min_length=30, do_sample=False))

[{'summary_text': 'This bag can hold a ton of stuff, and it has so many pockets! Many of the pockets have elastic around the top, to better hold things like bottles and sippy cups. The included travel changing pad is a good size, and larger than it seems in the photos.'}]


In [9]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
summarized_reviews = []
for r in new_df['review_body']:
    tokens = tokenizer(r, truncation=True, padding="longest", return_tensors="pt")
    summary = model.generate(**tokens, max_length=100)
    summarized_reviews.append(tokenizer.decode(summary[0]))
new_df['summarized_review'] = summarized_reviews 

C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_14328\1340887313.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['summarized_review'] = summarized_reviews


In [10]:
print(new_df['summarized_review'][0])
print(new_df['review_body'][0])
new_df

I've just received my first Surface Pro 4.
I was really excited about getting this. However, it isn't compatible to my keyboard despite seeming so. Enclosed is pics for comparison.


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,summarized_review
0,en_0824773,product_en_0722697,reviewer_en_0755840,2,I was really excited about getting this. Howev...,Not applicable,en,pc,I've just received my first Surface Pro 4.
1,en_0451671,product_en_0821454,reviewer_en_0028491,4,A little snug but I like them and I will go up...,Four Stars,en,shoes,These are my first pair of trainers and they a...
2,en_0582960,product_en_0067606,reviewer_en_0276540,4,I put both sets in a hanging glass candle hold...,"Pretty and functional, but one stopped working",en,lawn_and_garden,I've been using these lights for about six mon...
3,en_0015995,product_en_0700961,reviewer_en_0686612,1,I couldn't use this product. The packaging was...,I couldn't use this product. The packaging was...,en,shoes,I'm not a fan of this product.
4,en_0226240,product_en_0932759,reviewer_en_0565732,4,Highly recommend. The story of the youngest br...,Her Monster,en,digital_ebook_purchase,A wonderful book.
...,...,...,...,...,...,...,...,...,...
95,en_0249743,product_en_0595890,reviewer_en_0413731,3,"The scent is nice, I expected the consistency ...",It's ok,en,beauty,This is my first time using this product and I...
96,en_0269088,product_en_0587944,reviewer_en_0847314,2,They sent the wrong scent in a beat up package.,wrong thing sent,en,drugstore,A company in the UK has apologised after sendi...
97,en_0563784,product_en_0778840,reviewer_en_0736076,5,The duffel bag is of good quality but weighs v...,Good bag but heavy,en,luggage,This is one of the best bags I have ever owned.
98,en_0771345,product_en_0906567,reviewer_en_0639063,2,Bought this in December and it broke when I op...,Not durable,en,pc,This is one of the worst phones I have ever ow...


## DISTILBERT

In [11]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
print(summarizer(original_text, max_length=130, min_length=30, do_sample=False))

[{'summary_text': ' The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building . It was the first structure to reach a height of 300 metres . It is now taller than the Chrysler Building by 5.2 metres (17 ft) Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France .'}]


In [12]:
lengths = df["review_body"].str.len()
argmax = np.where(lengths == lengths.max())[0]
df.iloc[argmax]
df.iloc[argmax]['review_body']
#print(df.iloc[argmax[0]]['review_body'])
print(summarizer(df.iloc[argmax[0]]['review_body'], max_length=130, min_length=30, do_sample=False))

[{'summary_text': " The bag has a lot of space and elastic around the top of the main compartment . The main compartment zips on three sides, but it has pieces of fabric that stop it from flapping completely open . The bag is heavy, even empty . It's a good size - about the size of a backpack or small messenger bag . The color is nice, and the pattern is nice ."}]


In [13]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
summarized_reviews = []
for r in new_df['review_body']:
    summarized_reviews.append(summarizer(r, max_length=100, min_length=10, do_sample=False))
new_df['summarized_review'] = [summary[0]["summary_text"] for summary in summarized_reviews] 

Your max_length is set to 100, but you input_length is only 30. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 100, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 100, but you input_length is only 58. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Your max_length is set to 100, but you input_length is only 29. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Your max_length is set to 100, but you input_length is only 32. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 100, but you input_length is only 73. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 100, but you input_length is only 20. You might consider 

In [14]:
print(new_df['summarized_review'][0])
print(new_df['review_body'][0])
new_df

 I was really excited about getting this . However, it isn't compatible to my keyboard despite seeming so . Enclosed is pics for comparison .
I was really excited about getting this. However, it isn't compatible to my keyboard despite seeming so. Enclosed is pics for comparison.


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,summarized_review
0,en_0824773,product_en_0722697,reviewer_en_0755840,2,I was really excited about getting this. Howev...,Not applicable,en,pc,I was really excited about getting this . How...
1,en_0451671,product_en_0821454,reviewer_en_0028491,4,A little snug but I like them and I will go up...,Four Stars,en,shoes,A little snug but I like them and I will go u...
2,en_0582960,product_en_0067606,reviewer_en_0276540,4,I put both sets in a hanging glass candle hold...,"Pretty and functional, but one stopped working",en,lawn_and_garden,"After about 6 months, one set no longer turns..."
3,en_0015995,product_en_0700961,reviewer_en_0686612,1,I couldn't use this product. The packaging was...,I couldn't use this product. The packaging was...,en,shoes,The packaging was inadequate and the polish a...
4,en_0226240,product_en_0932759,reviewer_en_0565732,4,Highly recommend. The story of the youngest br...,Her Monster,en,digital_ebook_purchase,The story of the youngest brother in the crim...
...,...,...,...,...,...,...,...,...,...
95,en_0249743,product_en_0595890,reviewer_en_0413731,3,"The scent is nice, I expected the consistency ...",It's ok,en,beauty,The consistency is more oily and creamy but i...
96,en_0269088,product_en_0587944,reviewer_en_0847314,2,They sent the wrong scent in a beat up package.,wrong thing sent,en,drugstore,They sent the wrong scent in a beat up packag...
97,en_0563784,product_en_0778840,reviewer_en_0736076,5,The duffel bag is of good quality but weighs v...,Good bag but heavy,en,luggage,The duffel bag is of good quality but weighs ...
98,en_0771345,product_en_0906567,reviewer_en_0639063,2,Bought this in December and it broke when I op...,Not durable,en,pc,Bought this in December and it broke when I o...


## GPT-2

In [15]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2')
inputs=tokenizer.batch_encode_plus([original_text],return_tensors='pt',max_length=100)
summary_ids=model.generate(inputs['input_ids'],early_stopping=True)
GPT_summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
print(GPT_summary)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\marko.vasilic\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- 

The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to be


In [16]:
lengths = df["review_body"].str.len()
argmax = np.where(lengths == lengths.max())[0]
df.iloc[argmax]
df.iloc[argmax]['review_body']
#print(df.iloc[argmax[0]]['review_body'])
inputs=tokenizer.batch_encode_plus([df.iloc[argmax[0]]['review_body']],return_tensors='pt',max_length=100)
summary_ids=model.generate(inputs['input_ids'],early_stopping=True)
GPT_summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
print(GPT_summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 100, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Pros. A LOT of space. This bag can hold a ton of stuff, and it has so many pockets! Many of the pockets have elastic around the top, to better hold things like bottles and sippy cups. The main compartment has pockets all the way around the inside, and the two small pockets on the outside on either end of the bag also have elastic that allows them to expand and hold the items in securely.. I easily fit everything that's in my current diaper bag, into this bag


In [17]:
summarized_reviews = []
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2')
for r in new_df['review_body']:
    inputs=tokenizer.batch_encode_plus([original_text],return_tensors='pt',max_length=100)
    summary_ids=model.generate(inputs['input_ids'],early_stopping=True)
    summarized_reviews.append(tokenizer.decode(summary_ids[0],skip_special_tokens=True))
new_df['summarized_review'] = summarized_reviews 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 100, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:5025

In [18]:
print(new_df['summarized_review'][0])
print(new_df['review_body'][0])
new_df

The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to be
I was really excited about getting this. However, it isn't compatible to my keyboard despite seeming so. Enclosed is pics for comparison.


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,summarized_review
0,en_0824773,product_en_0722697,reviewer_en_0755840,2,I was really excited about getting this. Howev...,Not applicable,en,pc,"The tower is 324 metres (1,063 ft) tall, about..."
1,en_0451671,product_en_0821454,reviewer_en_0028491,4,A little snug but I like them and I will go up...,Four Stars,en,shoes,"The tower is 324 metres (1,063 ft) tall, about..."
2,en_0582960,product_en_0067606,reviewer_en_0276540,4,I put both sets in a hanging glass candle hold...,"Pretty and functional, but one stopped working",en,lawn_and_garden,"The tower is 324 metres (1,063 ft) tall, about..."
3,en_0015995,product_en_0700961,reviewer_en_0686612,1,I couldn't use this product. The packaging was...,I couldn't use this product. The packaging was...,en,shoes,"The tower is 324 metres (1,063 ft) tall, about..."
4,en_0226240,product_en_0932759,reviewer_en_0565732,4,Highly recommend. The story of the youngest br...,Her Monster,en,digital_ebook_purchase,"The tower is 324 metres (1,063 ft) tall, about..."
...,...,...,...,...,...,...,...,...,...
95,en_0249743,product_en_0595890,reviewer_en_0413731,3,"The scent is nice, I expected the consistency ...",It's ok,en,beauty,"The tower is 324 metres (1,063 ft) tall, about..."
96,en_0269088,product_en_0587944,reviewer_en_0847314,2,They sent the wrong scent in a beat up package.,wrong thing sent,en,drugstore,"The tower is 324 metres (1,063 ft) tall, about..."
97,en_0563784,product_en_0778840,reviewer_en_0736076,5,The duffel bag is of good quality but weighs v...,Good bag but heavy,en,luggage,"The tower is 324 metres (1,063 ft) tall, about..."
98,en_0771345,product_en_0906567,reviewer_en_0639063,2,Bought this in December and it broke when I op...,Not durable,en,pc,"The tower is 324 metres (1,063 ft) tall, about..."


## T5

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
tokenizer = AutoTokenizer.from_pretrained('t5-small', model_max_length=1500)
tokens_input = tokenizer.encode("summarize: "+ original_text, return_tensors='pt', max_length=tokenizer.model_max_length, truncation=True)
summary_ids = model.generate(tokens_input, min_length=10, max_length=100, length_penalty=20, num_beams=2)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

the Eiffel Tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. its base is square, measuring 125 metres (410 ft) on each side. it was the first structure to reach a height of 300 metres. it is now taller than the Chrysler Building by 5.2 metres (17 ft) it is the second tallest free-standing structure in France after the


In [20]:
lengths = df["review_body"].str.len()
argmax = np.where(lengths == lengths.max())[0]
df.iloc[argmax]
df.iloc[argmax]['review_body']
#print(df.iloc[argmax[0]]['review_body'])
tokens_input = tokenizer.encode("summarize: "+ df.iloc[argmax[0]]['review_body'], return_tensors='pt', max_length=tokenizer.model_max_length, truncation=True)
summary_ids = model.generate(tokens_input, min_length=10, max_length=100, length_penalty=20, num_beams=2)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

the main compartment has pockets all the way around the inside, and the two small pockets on the outside on either end of the bag also have elastic that allows them to expand and hold the items in securely. the main compartment zips on three sides, but it has pieces of fabric that stop it from flapping completely open. it's as wide as the bag is, so you don't have to worry about setting it down someplace that's bound to be dirty 


In [21]:
summarized_reviews = []
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
tokenizer = AutoTokenizer.from_pretrained('t5-small', model_max_length=1500)
for r in new_df['review_body']:
    tokens_input = tokenizer.encode("summarize: "+ r, return_tensors='pt', max_length=tokenizer.model_max_length, truncation=True)
    summary_ids = model.generate(tokens_input, min_length=10, max_length=100, length_penalty=20, num_beams=2)
    summarized_reviews.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))
new_df['summarized_review'] = summarized_reviews 
new_df

C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_14328\3683161451.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['summarized_review'] = summarized_reviews


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,summarized_review
0,en_0824773,product_en_0722697,reviewer_en_0755840,2,I was really excited about getting this. Howev...,Not applicable,en,pc,i was really excited about getting this. but i...
1,en_0451671,product_en_0821454,reviewer_en_0028491,4,A little snug but I like them and I will go up...,Four Stars,en,shoes,a little snug but I like them and I will go up...
2,en_0582960,product_en_0067606,reviewer_en_0276540,4,I put both sets in a hanging glass candle hold...,"Pretty and functional, but one stopped working",en,lawn_and_garden,the light is soft and warm. It's not quite bri...
3,en_0015995,product_en_0700961,reviewer_en_0686612,1,I couldn't use this product. The packaging was...,I couldn't use this product. The packaging was...,en,shoes,the packaging was inadequate and the polish ar...
4,en_0226240,product_en_0932759,reviewer_en_0565732,4,Highly recommend. The story of the youngest br...,Her Monster,en,digital_ebook_purchase,the story of the youngest brother in the crime...
...,...,...,...,...,...,...,...,...,...
95,en_0249743,product_en_0595890,reviewer_en_0413731,3,"The scent is nice, I expected the consistency ...",It's ok,en,beauty,"the scent is nice, I expected the consistency ..."
96,en_0269088,product_en_0587944,reviewer_en_0847314,2,They sent the wrong scent in a beat up package.,wrong thing sent,en,drugstore,they sent the wrong scent in a beat up package...
97,en_0563784,product_en_0778840,reviewer_en_0736076,5,The duffel bag is of good quality but weighs v...,Good bag but heavy,en,luggage,the duffel bag is of good quality but weighs v...
98,en_0771345,product_en_0906567,reviewer_en_0639063,2,Bought this in December and it broke when I op...,Not durable,en,pc,Bought this in December and it broke when I op...
